In [1]:
import torch
from torch.utils.data import DataLoader
from dataset import *
from modules import *
from train import *

In [2]:
train_set, val_set, test_set = generate_adni_datasets(datasplit=0.1)

train_loader = DataLoader(train_set, shuffle=True, batch_size=128)
val_loader = DataLoader(val_set, shuffle=True, batch_size=128)
test_loader = DataLoader(test_set, shuffle=False, batch_size=128)

[('C:/Users/Jun Khai/Documents/Uni/Year 5 Sem 2/PatternAnalysis-2023/recognition/45816967_ADNI_Vision_Transformer/data/ADNI_AD_NC_2D/AD_NC/train/AD/*', 1), ('C:/Users/Jun Khai/Documents/Uni/Year 5 Sem 2/PatternAnalysis-2023/recognition/45816967_ADNI_Vision_Transformer/data/ADNI_AD_NC_2D/AD_NC/train/NC/*', 0)]
[('C:/Users/Jun Khai/Documents/Uni/Year 5 Sem 2/PatternAnalysis-2023/recognition/45816967_ADNI_Vision_Transformer/data/ADNI_AD_NC_2D/AD_NC/test/AD/*', 1), ('C:/Users/Jun Khai/Documents/Uni/Year 5 Sem 2/PatternAnalysis-2023/recognition/45816967_ADNI_Vision_Transformer/data/ADNI_AD_NC_2D/AD_NC/test/NC/*', 0)]
[('C:/Users/Jun Khai/Documents/Uni/Year 5 Sem 2/PatternAnalysis-2023/recognition/45816967_ADNI_Vision_Transformer/data/ADNI_AD_NC_2D/AD_NC/train/AD/*', 1), ('C:/Users/Jun Khai/Documents/Uni/Year 5 Sem 2/PatternAnalysis-2023/recognition/45816967_ADNI_Vision_Transformer/data/ADNI_AD_NC_2D/AD_NC/train/NC/*', 0)]
[('C:/Users/Jun Khai/Documents/Uni/Year 5 Sem 2/PatternAnalysis-2023/

In [3]:
model = ViT(patch_size=16, num_transformer_layers=12, embedding_dim=256, mlp_size=256, num_heads=16).to(device)
summary(model=model, 
				input_size=(128, 1, 224, 224),
				col_names=["input_size", "output_size", "num_params", "trainable"],
				col_width=20,
				row_settings=["var_names"]
	)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ViT (ViT)                                [128, 1, 192, 192]   [128, 2]             66,048               True
├─PatchEmbedding (patch_embedding)       [128, 1, 192, 192]   [128, 256, 256]      --                   True
│    └─Conv2d (patcher)                  [128, 1, 192, 192]   [128, 256, 16, 16]   37,120               True
│    └─Flatten (flatten)                 [128, 256, 16, 16]   [128, 256, 256]      --                   --
├─Dropout (embedding_dropout)            [128, 257, 256]      [128, 257, 256]      --                   --
├─Sequential (transformer_encoder)       [128, 257, 256]      [128, 257, 256]      --                   True
│    └─TransformerEncoderLayer (0)       [128, 257, 256]      [128, 257, 256]      395,776              True
│    └─TransformerEncoderLayer (1)       [128, 257, 256]      [128, 257, 256]      395,776              True
│    └─Transformer

In [4]:

train(model, train_loader, val_loader, n_epochs=5, lr = 0.000025, version_prefix="vit11")

Using device:  cuda (NVIDIA GeForce RTX 3060 Ti)


Epoch 1/5 loss: 0.68
Train accuracy: 55.12%


Validation: 100%|██████████| 17/17 [00:09<00:00,  1.72it/s]


Val loss: 0.68
Val accuracy: 51.50%


Epoch 2 in training:  47%|████▋     | 71/152 [09:31<11:03,  8.19s/it]

In [ ]:
model2 = torch.load("models/vit2_model_12_0.5835193874020326.pth")

In [ ]:
with torch.no_grad():
    correct, total = 0, 0
    test_loss = 0.0
    for batch in tqdm(test_loader, desc="Testing"):
        x, y = batch
        x, y = x.to(device), y.to(device)
        y_hat = model2(x)
        loss = criterion(y_hat, y)
        test_loss += loss.detach().cpu().item() / len(test_loader)

        correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()
        total += len(x)
    print(f"Test loss: {test_loss:.2f}")
    print(f"Test accuracy: {correct / total * 100:.2f}%")

Testing:   0%|          | 0/71 [00:00<?, ?it/s]


TypeError: 'collections.OrderedDict' object is not callable